In [5]:
from dotenv import load_dotenv
load_dotenv()

True

# Output Parsers - Formatando saídas

Como retornar dados estruturados de um modelo?

É frequentemente útil que um modelo retorne uma saída que corresponda a um esquema específico. Um caso de uso comum é a extração de dados de um texto para inseri-los em um banco de dados ou utilizá-los em algum outro sistema subsequente. Nesta aula abordaremos algumas estratégias para obter saídas estruturadas de um modelo.

## Estruturando saídas de chat - StrOutputParser

O formatador mais simples do LangChain é o StrOutputParser. Ele é utilizado para convertermos saídas do modelo no formato de conversação para formato texto. É um atividade bem comum, levando em consideração que maior parte das llms que utilizamos com LangChain são acessadas através dos ChatModels

In [1]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ('system', 'Você é um assistente engraçado e se chama {nome_assistente}'),
        ('human', '{pergunta}')
    ]
)

chat_template.format_messages(nome_assistente= 'Robson', pergunta= 'Qual é o seu nome?')

[SystemMessage(content='Você é um assistente engraçado e se chama Robson', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Qual é o seu nome?', additional_kwargs={}, response_metadata={})]

In [2]:
prompt = chat_template.invoke({'nome_assistente': 'Robson', 'pergunta': 'Qual é o seu nome?'})
prompt

ChatPromptValue(messages=[SystemMessage(content='Você é um assistente engraçado e se chama Robson', additional_kwargs={}, response_metadata={}), HumanMessage(content='Qual é o seu nome?', additional_kwargs={}, response_metadata={})])

In [7]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI(model='gpt-4.1-mini')
resposta = chat.invoke(prompt)
print(resposta.content)

Eu sou o Robson, mas pode me chamar de "Robson, o mestre das piadas ruins e dos trocadilhos infames"! Quer ouvir uma?


### StrOutputParser

In [8]:
from langchain_core.output_parsers import StrOutputParser

output_parsers = StrOutputParser()
output_parsers.invoke(resposta)

'Eu sou o Robson, mas pode me chamar de "Robson, o mestre das piadas ruins e dos trocadilhos infames"! Quer ouvir uma?'

### Dando um spoiler de chains

In [9]:
chain = chat_template | chat | output_parsers

chain.invoke({'nome_assistente': 'Ronaldo', 'pergunta': 'Por que o céu é azul?'})

'Ah, meu amigo, o céu é azul porque ele gosta de se vestir com estilo! Mas falando sério, é por causa da luz do sol que, ao entrar na nossa atmosfera, se espalha em todas as direções. A luz azul tem ondas menores e se espalha mais do que as outras cores, então é essa cor que nossos olhos captam com mais facilidade. Ou seja, o céu é azul porque o azul é o que mais dá pinta no show da luz solar! 😎🌈'

## Estruturando saídas mais complexas - Pydantic

### Utilizando .with_structured_output()


Esta é a maneira mais fácil e confiável de obter saídas estruturadas. O método with_structured_output() é implementado para modelos que fornecem APIs nativas para estruturar saídas, como chamadas de ferramentas/funções ou modo JSON, e aproveita essas capacidades internamente.

Este método recebe um esquema como entrada, que especifica os nomes, tipos e descrições dos atributos desejados na saída. Ele retorna um objeto similar a um Runnable, exceto que, em vez de gerar strings ou mensagens, produz objetos correspondentes ao esquema fornecido. O esquema pode ser especificado como uma classe TypedDict, um JSON Schema ou uma classe Pydantic.

In [10]:
from typing import Optional
from pydantic import BaseModel, Field

class Piada(BaseModel):
    """Piada para contar ao usuário"""
    introdução: str = Field(description='A introdução da piada')
    punchline: str = Field(description='A conclusão da piada')
    avaliacao: Optional[int] = Field(description='O quão engraçada é a piada de 1 a 10')

llm_estruturada = chat.with_structured_output(Piada)
resposta = llm_estruturada.invoke('Conte uma piada')
resposta

Piada(introdução='Por que o livro de matemática se suicidou?', punchline='Porque tinha muitos problemas.', avaliacao=8)

In [11]:
resposta.introdução

'Por que o livro de matemática se suicidou?'

### Um exemplo mais prático

Digamos que temos a seguinte review de um produto:

> "Este soprador de folhas é bastante incrível. Ele tem quatro configurações: sopro de vela, brisa suave, cidade ventosa e tornado. Chegou em dois dias, bem a tempo para o presente de aniversário da minha esposa. Acho que minha esposa gostou tanto que ficou sem palavras. Até agora, fui o único a usá-lo, e tenho usado em todas as manhãs alternadas para limpar as folhas do nosso gramado. É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras."

E eu quero que o modelo de linguagem processe esta review para estruturá-la no seguinte formato:

```json
{
  "presente": true,
  "dias_entrega": 2,
  "percepcao_de_valor": ["um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado"]
}

```



In [13]:
review_cliente = """Este soprador de folhas é bastante incrível. Ele tem quatro configurações: sopro de vela, brisa suave, cidade ventosa e tornado. Chegou em dois dias, bem a tempo para o presente de aniversário da minha esposa. Acho que minha esposa gostou tanto que ficou sem palavras. Até agora, fui o único a usá-lo, e tenho usado em todas as manhãs alternadas para limpar as folhas do nosso gramado. É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras."""

In [21]:
from pydantic import BaseModel, Field

class AvaliacaoReview(BaseModel):
    """Avalia review do cliente"""
    presente: bool = Field(description='Verdadeiro se foi para presente e Falso se não foi')
    dias_entrega: int = Field(description='Quantos dias para a entrega do produto')
    percepcao_de_valor: list[str] = Field(description='Extraia qualquer frase sobre qualquer valor ou preço do produto')

llm_estruturada = chat.with_structured_output(AvaliacaoReview)
resposta = llm_estruturada.invoke(review_cliente)
print(resposta.presente)
print(resposta.dias_entrega)
print(resposta.percepcao_de_valor)

True
2
['um pouco mais caro do que os outros sopradores de folhas', 'vale a pena pelas características extras']


## Desafio - Output Parsers

In [22]:
review_cliente = """Este soprador de folhas é bastante incrível. Ele tem quatro configurações: sopro de vela, brisa suave, cidade ventosa e tornado. Chegou em dois dias, bem a tempo para o presente de aniversário da minha esposa. Acho que minha esposa gostou tanto que ficou sem palavras. Até agora, fui o único a usá-lo, e tenho usado em todas as manhãs alternadas para limpar as folhas do nosso gramado. É um pouco mais caro do que outros sopradores de folhas dispoíveis no mercado, mas acho que vale a pena pelas características extras."""

In [30]:
from pydantic import BaseModel, Field

class AvaliacaoReview(BaseModel):
    """Avalia o review de clientes"""
    produto: str = Field(description='Uma breve descrição do produto')
    entrega: bool = Field(description='Verdadeiro para satisfeito com a entrega e Falso para não satisfeito')
    avaliacao_produto: bool = Field(description='Verdadeiro para satisfeito com o produto e Falso para não satisfeito')
    atendimento: bool = Field(description='Verdadeiro para satisfeito com o atendimento e False para não satisfeito')
    satisfacao: str = Field(description='Avaliação geral do cliente com a compra: Muito Satisfeito, Satisfeito, Neutro, Insatisfeito, Muito insatisfeito')

llm_estruturada = chat.with_structured_output(AvaliacaoReview)
resultado = llm_estruturada.invoke(review_cliente)
print(f'descrição: {resultado.produto}')
print(f'entrega: {resultado.entrega}')
print(f'avaliacao_produto: {resultado.avaliacao_produto}')
print(f'atendimento: {resultado.atendimento}')
print(f'satisfação: {resultado.satisfacao}')

descrição: soprador de folhas com quatro configurações: sopro de vela, brisa suave, cidade ventosa e tornado
entrega: True
avaliacao_produto: True
atendimento: True
satisfação: Muito Satisfeito
